In [1]:
import carball
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
%matplotlib inline

In [2]:
REPLAYS_DIR = 'replays'
replays = glob.glob(os.path.join(REPLAYS_DIR, '*.replay'))
replays[:2]

['replays/6765D43F454D5BB0522EE49645C0FEF1.replay',
 'replays/8E76D5E74AD2B7580B43BABA1FF7D586.replay']

In [161]:
working_replay = replays[1]
manager = carball.analyze_replay_file(working_replay, "replay.json")
proto = manager.protobuf_game
dataframe = manager.data_frame

Player RLCS Admin as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Admin as player has no team.
Player RLCS Admin as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Admin as player has no team.
Player RLCSobserver3 as player has no MatchScore.
Score is not found for player
Ignoring player: RLCSobserver3 as player has no team.
Player RLCSobserver2 as player has no MatchScore.
Score is not found for player
Ignoring player: RLCSobserver2 as player has no team.
/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/stats/boost/boost.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  player_data_frame['delta'] = game.frames.delta


In [139]:

num_columns = 151
class GoalPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_columns, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, game_state):
        return self.layers(game_state)

In [162]:
def get_train_test_data(proto, dataframe):
    # Only from the perspective of the blue team (we'll switch them up)
    players = dataframe.columns.levels[0]
    columns = dataframe.columns.levels[1]
    teams = proto.teams
    team_map = {}
    for i, team in enumerate(teams):
        for player in team.player_ids:
            team_map[player.id] = int(team.is_orange)
    goal_teams = []
    for goal in proto.game_metadata.goals:
        goal_teams.append(team_map[goal.player_id.id])
        
    # Get goal number
    goal_number = dataframe['game']['goal_number'].dropna()
    goal_team = goal_number.apply(lambda x: goal_teams[int(x)])
    #goal_frame = goal_number.apply(lambda x: goals[int(x)].frame_number)
    df = dataframe.copy()
    df['game', 'goal_team'] = goal_team.rename('goal_team')
    #df['game', 'goal_frame'] = goal_frame.rename('goal_frame')
    df = df.dropna(subset=[('game','goal_team')])
    actors = players[:-1]
    input_data = df[actors]
    if len(input_data.columns) != num_columns:
        return None, None, None, None
    
    # Set up data
    input = input_data.fillna(0).astype(float)
    output = df['game']['goal_team'].values.reshape((-1, 1))
    rand = np.random.rand(output.shape[0])
    threshold = rand < 0.8
    input_train, input_test = input[threshold].values, input[~threshold].values
    output_train, output_test = output[threshold], output[~threshold]
    
    return input_train, input_test, output_train, output_test
    

def train_on_proto_df(proto, df, model):
    input_train, input_test, output_train, output_test = get_train_test_data(proto, df)
    if input_train is None:
        return
    loss = nn.BCELoss()
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
    input_tensor = torch.from_numpy(input_train).float().cuda()
    output_tensor = torch.from_numpy(output_train).float().cuda()
    for x in range(1000):
        predicted = model(input_tensor)
        loss_val = loss(predicted, output_tensor)
        loss_val.backward()
        opt.step()
        if x % 100 == 0:
            print('Loss:', loss_val)

        if x % 500 == 0:
            test_output = model(torch.from_numpy(input_test).float().cuda())
            output = test_output.cpu().detach().numpy()
            print('Accuracy', (output == output_test).sum() / output.shape[0])

In [163]:
import requests
import gzip
from carball.analysis.analysis_manager import PandasManager
from carball.analysis.utils.proto_manager import ProtobufManager

import io
BASE_URL = 'https://calculated.gg/api/v1/'

def get_replay_list(num=50):
    r = requests.get(BASE_URL + 'replays?key=1&minrank=19&teamsize=3')
    return r.json()['data']

def get_pandas(id_):
    url = BASE_URL + 'parsed/{}.replay.gzip?key=1'.format(id_)
    r = requests.get(url)
    gzip_file = gzip.GzipFile(fileobj=io.BytesIO(r.content), mode='rb')

    pandas_ = PandasManager.safe_read_pandas_to_memory(gzip_file)
    return pandas_

def get_proto(id_):
    url = BASE_URL + 'parsed/{}.replay.pts?key=1'.format(id_)
    r = requests.get(url)
#     file_obj = io.BytesIO()
#     for chunk in r.iter_content(chunk_size=1024):
#         if chunk: # filter out keep-alive new chunks
#             file_obj.write(chunk)
    proto = ProtobufManager.read_proto_out_from_file(io.BytesIO(r.content))
    return proto

In [166]:

model = GoalPredictor().cuda()
for replay in get_replay_list():
    id_ = replay['hash']
    
    df = get_pandas(id_)
    proto = get_proto(id_)
    if df is None or proto is None:
        continue
    try:
        train_on_proto_df(proto, df, model)
    except:
        pass

Loss: tensor(11.4908, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.5481081081081081
Loss: tensor(4.4078, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(4.1749, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(4.1552, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(4.4888, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(4.4966, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.8551351351351352
Loss: tensor(4.4221, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(4.4731, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(5.5403, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.7141, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.8750, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.711453744493392
Loss: tensor(7.0601, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss

Failure to read pandas [] from memory: Struct error
Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 49, in get_array
    starting_byte = struct.unpack('i', chunk)[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 27, in safe_read_pandas_to_memory
    return PandasManager.read_numpy_from_memory(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 40, in read_numpy_from_memory
    array = read_array_from_file(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 36, in read_array_fro

Loss: tensor(7.2323, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.7338086749851456
Loss: tensor(7.2600, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.2362, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.2124, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.1886, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.1608, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.738562091503268
Loss: tensor(7.1052, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.1052, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.1052, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(7.1052, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(21.0392, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.23328067403896788
Loss: tensor(21.0392, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Lo

Failure to read pandas [] from memory: Struct error
Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 49, in get_array
    starting_byte = struct.unpack('i', chunk)[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 27, in safe_read_pandas_to_memory
    return PandasManager.read_numpy_from_memory(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 40, in read_numpy_from_memory
    array = read_array_from_file(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 36, in read_array_fro

Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.0
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.0
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(11.0725, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.6097838452787259
Loss: tensor(11.0725, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(11.0725, 

Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.7642045454545454
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(6.6878, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Accuracy 0.0
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Loss: tensor(27.6310, device='cuda:0', gra

In [167]:
# Still need to sort by team etc